In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pdb
from astropy.table import QTable
import shutil
%matplotlib inline

In [13]:
# lets couple this shit
i = 0 
n = 25000

eqorb_dir = './final_data/runs_eqorb/'
eq_only_dir = './final_data/runs_eq_only/'

def add_atmosphere(dMass, dTidalQ, dK2):
    dPresXUV = float(np.random.uniform(0.1,10,1))
    dThermTemp = float(np.random.uniform(880,3000,1))
    atmmassfrac = float(np.random.uniform(0.01,0.1,1))
    dEnvelopeMass = float(dMass * atmmassfrac * -1)
    dAtmXAbsEffH = float(np.random.uniform(0.1,0.6,1))
    dAtmGasConst = float(np.random.uniform(3600,4157,1))
    dSatXUVTime = float(np.random.uniform(80e6,120e6,1))
    dTidalQRock = float(np.random.uniform(30,301,1))
    dK2Rock = 0.3
    dTidalQEnv = dTidalQ
    dK2Env = dK2
    bEnvTides = 1
    return (dPresXUV, dThermTemp, dEnvelopeMass, dAtmXAbsEffH, dAtmGasConst, dSatXUVTime,
            dTidalQRock, dK2Rock, dTidalQEnv, dK2Env)

def write_atmesc_stuff(body_content, src_dir, dest_dir,bdistorb):
    # here body_content is the b/c.in file read in using .readlines()
    # first lets create the destination directory:
    # if the vpl/star files dont already exist in the destination directory, we copy them over
    if (os.path.isfile(dest_dir+'/vpl.in') == 0): 
        os.makedirs(dest_dir)
        shutil.copy(src_dir + "/vpl.in",dest_dir+"/vpl.in")
    
    # we change dTidalQ to dTidalQEnv and dK2 to dK2Env
    name = body_content[0].split('\t\t')[1][:-1]
    if name == 'b' and bdistorb == 0:
        tidalq_line = 17
        k2_line = 18
    elif name == 'c':
        tidalq_line = 17
        k2_line = 18
    elif name == 'b':
        tidalq_line = 19
        k2_line = 20

    body_content[tidalq_line] = body_content[tidalq_line].replace('dTidalQ\t','dTidalQEnv')
    body_content[k2_line] = body_content[k2_line].replace('dK2','dK2Env')
    
    if bdistorb:
        modules = 'eqtide distorb'
    else:
        modules = 'eqtide'
    
    body_content[1] = body_content[1].replace(modules,modules+' atmesc')
    
    atmesc_content = ("\nsPlanetRadiusModel\t" + "lehmer" +
                      "\ndEnvelopeMass\t\t" + str(dEnvelopeMass) + 
                      "\ndPresXUV\t\t" + str(dPresXUV) +
                      "\ndThermTemp\t\t" + str(dThermTemp) + 
                      "\ndAtmXAbsEffH\t\t" + str(datmXAbsEffH) + 
                      "\ndAtmGasConst\t\t" + str(dAtmGasConst) + 
                      "\ndTidalQRock\t\t" + str(dTidalQRock) + 
                      "\ndK2Rock\t\t\t" + str(dK2Rock) + 
                      "\nbHaltEnvelopeGone\t\t" + "0" + 
                      "\n")
    
    body_content.insert(-2,atmesc_content)
    body_content[-1] = body_content[-1][:-1] + " -Mass -EnvelopeMass TidalQ FXUV" + body_content[-1][-1]
    body_content = "".join(body_content)
    body_file = open(dest_dir+'/'+name+'.in','w')
    body_file.write(body_content)
    body_file.close()

def write_stellar(sim_dir, b_atm, c_atm):
    if (b_atm or c_atm):
        src_star_file = open(sim_dir+'/star.in','r')
        star_content = src_star_file.readlines()
        star_content[1] = star_content[1].replace('eqtide','eqtide stellar')
        star_content.insert(-2,"\ndSatXUVTime\t"+str(dSatXUVTime))
        star_content = "".join(star_content)
        dest_star_file = open(dest_dir+sim_number+'/star.in','w')
        dest_star_file.write(star_content)
        dest_star_file.close()

while i <= n-1:
    sim_number = "%05i"%i
    sim_dir = eqorb_dir + sim_number
    dest_dir = './eqorbmesc/'
    # lets read in the input files
    #eqorb first
    c_atm = 0 # control to determine if an atmosphere was added
    c_file = sim_dir + "/c.in"
    c_file = open(c_file,'r')
    c_content = c_file.readlines()
    c_tidalq = float(c_content[17].split('\t\t')[1][:-1])
    c_mass = float(c_content[4].split('\t\t')[1][:-1])*-1
    c_k2 = float(c_content[18].split('\t\t')[1][:-1])
    # add an atmosphere to c
    if c_tidalq > 400:
        c_atm = 1
        (dPresXUV, dThermTemp, dEnvelopeMass, datmXAbsEffH, dAtmGasConst, dSatXUVTime,
         dTidalQRock, dK2Rock, dTidalQEnv, dK2Env) = add_atmosphere(c_mass, c_tidalq, c_k2)
        write_atmesc_stuff(c_content, sim_dir, dest_dir+sim_number,bdistorb=1)
    c_file.close()
    
    
    b_atm = 0 # control to determine if an atmosphere was added
    b_file = sim_dir + "/b.in"
    b_file = open(b_file,'r')
    b_content = b_file.readlines()
    b_tidalq = float(b_content[19].split('\t\t')[1][:-1])
    b_mass = float(b_content[4].split('\t\t')[1][:-1])*-1
    b_k2 = float(b_content[20].split('\t\t')[1][:-1])
    # add an atmosphere to b
    if b_tidalq > 400:
        b_atm = 1
        (dPresXUV, dThermTemp, dEnvelopeMass, datmXAbsEffH, dAtmGasConst, dSatXUVTime,
         dTidalQRock, dK2Rock, dTidalQEnv, dK2Env) = add_atmosphere(b_mass, b_tidalq, b_k2)
        write_atmesc_stuff(b_content, sim_dir, dest_dir+sim_number,bdistorb=1)
    b_file.close()
    
    # now we need to other files for distorb to work, and stellar
    if (b_atm==1 and c_atm==0):
        shutil.copy(sim_dir+'/c.in',dest_dir+sim_number+'/c.in')
        write_stellar(sim_dir, b_atm, c_atm)
    elif (b_atm == 0 and c_atm == 1):
        shutil.copy(sim_dir+'/b.in',dest_dir+sim_number+'/b.in')
        write_stellar(sim_dir, b_atm, c_atm)
    elif (b_atm == 1 and c_atm == 1):
        write_stellar(sim_dir, b_atm, c_atm)

    # now lets do the same stuff for eqtide_only
    sim_dir = eq_only_dir + sim_number
    dest_dir = './eqmesc/'
    b_atm = 0
    b_file = sim_dir+'/b.in'
    b_file = open(b_file,'r')
    b_content = b_file.readlines()
    b_tidalq = float(b_content[17].split('\t\t')[1][:-1])
    if b_tidalq > 400:
        # we've already added at atmosphere, so lets just use the previous values
        b_atm = 1
        write_atmesc_stuff(b_content,sim_dir,dest_dir+sim_number,bdistorb=0)
        write_stellar(sim_dir, b_atm, c_atm=0)
    b_file.close()
    print (sim_dir,end='\r')
    i += 1

In [ ]:
os.path.isfile('./final_data/runs_eqorb/00000/b.in') != 0

In [ ]:
# what things do we need to add for each planet that has an atmospheric tidalq
# eqtide+atmesc
# sPlanetRadiusModel = lehmer
# dthermtemp
# dpresxuv
# datmgasconst
# bEnvTides = 1
# dTidalQEnv
# dK2Env
# dTidalQRock
# dK2Rock



In [ ]:
'''
- plot of env mass, mass, semi, ecc, tidalq, for case env --> rock; 
- gen data sets for eqmesc, and eqorbmesc. 
'''

In [ ]:
# now lets make some figures to show the above distribtions, just to double check.
# but first how can I double check I've done everything right...
# that atmospheres in the above 

eqorbmesc_dir = './eqorbmesc/'
eqmesc_dir = './eqmesc/'

eqmesc_sims = os.listdir(eqmesc_dir)
eqorbmesc_sims = os.listdir(eqorbmesc_dir)




In [4]:
# now to do only atmesc cases for the inner planet
n = 25000

eq_only = './final_data/runs_eq_only/'

def add_atmosphere(dMass):
    dPresXUV = float(np.random.uniform(0.1,10,1))
    dThermTemp = float(np.random.uniform(880,3000,1))
    atmmassfrac = float(np.random.uniform(0.01,0.1,1))
    dEnvelopeMass = float(dMass * atmmassfrac * -1)
    dAtmXAbsEffH = float(np.random.uniform(0.1,0.6,1))
    dAtmGasConst = float(np.random.uniform(3600,4157,1))
    dSatXUVTime = float(np.random.uniform(80e6,120e6,1))

    bEnvTides = 1
    return (dPresXUV, dThermTemp, dEnvelopeMass, dAtmXAbsEffH, dAtmGasConst, dSatXUVTime)


def write_atmesc(sim_dir, mass, ecc, semi, envmass, abseffh, thermtemp, presxuv, atmgasconst):
    body_content = ('sName\t\tb'+
                    '\nsaModules\t\t'+'atmesc'+
                    
                    '\n\ndMass\t\t' + str(mass*-1) + 
                    '\ndObliquity\t\t' + '23.5' + 
                    '\ndRadGyra\t\t' + '0.5' +
                    
                    '\n\ndEcc\t\t' + str(ecc) + 
                    '\ndSemi\t\t' + str(semi) + 
                    
                    '\n\nbHaltEnvelopeGone\t\t' + '1' +
                    '\ndEnvelopeMass\t\t' + str(envmass*-1) +
                    '\ndAtmXAbsEffH\t' + str(abseffh) + 
                    
                    '\n\nsPlanetRadiusModel\t'+'lehmer'+
                    '\ndThermTemp\t\t'+str(thermtemp)+
                    '\ndPresXUV\t\t' + str(presxuv) + 
                    '\ndAtmGasConst\t\t'+str(atmgasconst) + 
                    
                    '\n\nsaOutputOrder\t' + 'Time -Mass -EnvelopeMass' +
                    '\n')
    
    star_content = ('sName\t\t' + 'star' + 
                    '\nsaModules' + 'stellar' + 
                    '')
    
    
    
    


i=0
while i<=n-1:
    sim = "%05i"%i
    
    # open b
    b_atm = 0
    b_file = open(eq_only + sim + '/b.in','r')
    b_content = b_file.readlines()
    pdb.set_trace()
    b_mass = float(b_content[4].split('\t\t')[1][:-1])*-1
    b_semi = float(b_content[10].split('\t\t')[1][:-1])
    b_ecc = float(b_content[9].split('\t\t')[1][:-1])
    
    b_tidalq = float(b_content[17].split('\t\t')[1][:-1])
    if b_tidalq > 400:
        # add an atmosphere
        b_atm = 1
        dPresXUV, dThermTemp, dEnvelopeMass, dAtmXAbsEffH, dAtmGasConst, dSatXUVTime = add_atmosphere(b_mass)
        write_atmesc()
    i += 1

> <ipython-input-4-fa564d5a8f40>(54)<module>()
-> b_mass = float(b_content[4].split('\t\t')[1][:-1])*-1
(Pdb) b_content
['sName\t\tb\n', 'saModules\teqtide\n', 'sTideModel\tp2\n', '\n', 'dMass\t\t-11.4399233456\n', 'dRadius\t\t-3.97659149033\n', 'dObliquity\t45\n', 'dRadGyra\t0.5\n', '\n', 'dEcc\t\t0.147335238354\n', 'dSemi\t\t0.0795236578653\n', '\n', 'dInc\t\t0.0\n', 'dLongA\t\t197.329912893\n', 'dArgP\t\t312.39389086\n', '\n', 'bForceEqSpin\t1\n', 'dTidalQ\t\t5930204.56063\n', 'dK2\t\t0.3\n', 'saTidePerts\tstar\n', '\n', 'saOutputOrder\tTime Semim Ecce Obliquity SurfEnFluxEqtide\n']
(Pdb) b_content[10]
'dSemi\t\t0.0795236578653\n'
(Pdb) q


BdbQuit: 

In [12]:
final_data_dir = './runs_eqmesc/'

sim_list = np.array([])
atmmassfrac_list = np.array([])
for name in os.listdir(final_data_dir): 
    if os.path.isdir(final_data_dir+name): 
        sim_list = np.append(sim_list, name)
        b_file = open(final_data_dir+name+'/b.in','r')
        b_content = b_file.readlines()
        b_mass = float(b_content[4].split('\t\t')[1][:-1])*-1
        b_envmass = float(b_content[22].split('\t\t')[1][:-1])*-1
        b_atmmassfrac = b_envmass/b_mass
        atmmassfrac_list = np.append(atmmassfrac_list, b_atmmassfrac)
        
sim_list[np.argmin(atmmassfrac_list)]

'00299'